# <center>Intro to Hypothesis Testing</center>

## Part 1: our first hypothesis test

You find a brain in the human dissection lab, but you suspect it may belong to a new unknown species.

You decide that mass of brain is the only important feature. So you will measure its mass, then compare it to an online database of all human brain masses.

So you measure the mass of the brain, and record it:

In [35]:
sample_brain_mass = 1564.2 # grams

Then you download your dataset and load it into Python.

In [36]:
import numpy as np
brain_masses = np.load('human_brain_masses.npy')
print(brain_masses)

[1031.03862494 1132.34383661 1188.58290064 ... 1325.03535947 1342.84315573
 1578.74968743]


### __Does our sample brain come from a different *population* than the brains in this list of human brain masses?__  
*(it's worth noting, though okay to ignore for now, that we are treating these brain measurements as a __population__ and not a sample from a larger population–we'll deal with this later)*  

How can we answer that question?

<i>Intuitive approach: 

Is our brain smaller than the largest human brain?  
Is it larger than the smallest human brain?  
Is it larger or smaller than the average human brain?  
How many human brains are larger than our sample?  
How many are smaller? </i>

__Task__:  
Answer these questions using Python.

In [37]:
# <answer>

heaviest_brain_mass = np.max(brain_masses)
lightest_brain_mass = np.min(brain_masses)
average_brain_mass = np.mean(brain_masses)

if sample_brain_mass < heaviest_brain_mass:
    print('Our brain is lighter than the heaviest human brain.')

if sample_brain_mass > lightest_brain_mass:
    print('Our brain is heavier than the lightest human brain.')
    
if sample_brain_mass < average_brain_mass:
    print('Our brain is lighter than the average human brain.')
elif sample_brain_mass > average_brain_mass:
    print('Our brain is heavier than the average human brain.')

n_larger = np.sum(brain_masses > sample_brain_mass)
print(f'{n_larger} human brains are larger than our sample.')

n_smaller = np.sum(brain_masses < sample_brain_mass)
print(f'{n_smaller} human brains are smaller than our sample.')

# </answer>

Our brain is lighter than the heaviest human brain.
Our brain is heavier than the lightest human brain.
Our brain is heavier than the average human brain.
251206 human brains are larger than our sample.
6748793 human brains are smaller than our sample.


We could ask hundreds of such questions. Visualization can help speed this up.

__Task__:  
(1) plot a histogram of the human brain mass dataset  
(2) label our sample brain on that histogram

In [38]:
# <answer>

import matplotlib.pyplot as pl

pl.figure(figsize=(12,7))
fontsize = 'x-large'

# title this "population"
# show a histogram of all brain masses
hvals,hbins,hps = pl.hist(brain_masses, bins=500, histtype='step', linewidth=2, color='grey')
pl.xlabel('Brain mass (g)', fontsize=fontsize)
pl.ylabel('Number of brains', fontsize=fontsize)
pl.title('Human population brain masses', fontsize=fontsize, pad=60, color='grey')

Text(0.5,1,'Human population brain masses')

In [39]:
mean_human_brain_mass = np.mean(brain_masses)

pl.axvline(mean_human_brain_mass, color='darkblue')
pl.text(mean_human_brain_mass+20, 10000, 'Average\nbrain', color='darkblue', fontsize=fontsize)

Text(1312.02,10000,'Average\nbrain')

In [40]:
# display our sample brain mass on the histogram
pl.axvline(sample_brain_mass, color='darkmagenta')
pl.text(1580, 30000, 'Sample\nbrain', color='darkmagenta', fontsize=fontsize)

Text(1580,30000,'Sample\nbrain')

In [41]:
# text

delta = sample_brain_mass - mean_human_brain_mass 
pl.hlines(65000, mean_human_brain_mass, sample_brain_mass, color='darkgray')
pl.text((mean_human_brain_mass+sample_brain_mass)/2, 66000, f'{delta:0.2f} g', color='darkgray', fontsize=fontsize, ha='center')

hbins = (hbins[1:]+hbins[:-1])/2 # bin centers
fill_xs = hbins[hbins>=sample_brain_mass]
fill_ys = hvals[hbins>=sample_brain_mass]
pl.fill_between(fill_xs, fill_ys, color='grey')

# two-tailed
pl.hlines(65000, mean_human_brain_mass-delta, mean_human_brain_mass, color='darkgray')
pl.text(mean_human_brain_mass-delta/2, 66000, f'{delta:0.2f} g', color='darkgray', fontsize=fontsize, ha='center')
fill_xs = hbins[hbins<=mean_human_brain_mass-delta]
fill_ys = hvals[hbins<=mean_human_brain_mass-delta]
pl.fill_between(fill_xs, fill_ys, color='grey')

# </answer>

While it's not possible to be *certain* about the answer to our question, we can judge *how likely* it is that our brain came from this population distribution.

One measure of that is the fraction of brains that are __further from the mean__ than our sample is.

__Task:__  
(1) Compute the *fraction* of human brains with a mass larger than our sample brain.  
(2) Compute the *fraction* of human brains with a mass *further from the mean* than our sample brain.

In [42]:
# <answer>
fraction_larger = np.mean(brain_masses > sample_brain_mass)
print(f'{fraction_larger:0.2f} of all brains are larger than our sample.')

fraction_further = np.mean(np.abs(brain_masses-mean_human_brain_mass) > (sample_brain_mass-mean_human_brain_mass))
print(f'{fraction_further:0.2f} of all brains are further from the mean than our sample.')
# </answer>

0.04 of all brains are larger than our sample.
0.07 of all brains are further from the mean than our sample.


We could rephrase the latter fraction as "the probability of finding a brain with a more outlying mass than our sample."  
This is often referred to as a "p-value."

The first value is a "one-tailed" value, and the second is a "two-tailed" value. Unless you have strong reason to believe your sample could not have been extreme in other direction (in our case, lighter), then it makes most sense to use a two-tailed test.

### Generalizing the approach

If we can *assume* that the population underlying our data are *normally distributed*, then we can make that analysis very general.

(Recall the <a href="https://en.wikipedia.org/wiki/Central_limit_theorem">central limit theorem</a> for some basic intuition on the generality of normal distributions.)

To see why, let's visualize the basic properties of a normal distribution.

In [43]:
from scipy.stats import norm

xs = np.linspace(-4,4,1000)
ys = norm.pdf(xs, 0, 1)

pl.plot(xs, ys, color='k', lw=2)

for i,col in zip([3, 2.5, 2, 1], [.5, .4, .3, .2]):
    use = (xs>0) & (xs<i)
    pl.fill_between(xs[use], ys[use], color=pl.cm.Greys(col), lw=0)
    pl.annotate(f'{i} std', [np.max(xs[use])+.15,np.min(ys[use])+.01], color=pl.cm.Greys(col), ha='center')
    
# todo: add labels etc

To help illustrate the value of understanding this point, consider what happens when we choose to show our data in an arbitrary new unit of measurement.

In [34]:
fig,axs = pl.subplots(2,1,figsize=(5,7), sharex=True)

for ax,scale,unit in zip(axs, [1,0.035274], ['g','oz']):
    ax.hist(brain_masses*scale, color='k', bins=500, histtype='step')
    ax.set_xlabel(f'Brain mass ({unit})')
    ax.set_yticks([])
    ax.axvline(sample_brain_mass*scale, color='darkmagenta')

The data are identical, but if we didn't know that, it would not be immediately obvious how much of an "outlier" each one is, because the units are different.  

To get around this, we normalize our measurements by subtracting the mean and dividing by standard deviation to produce a __z-score__.  
<br>
<center>$z = \frac{x-\mu}{\sigma}$</center>

# use more intuitive symbols?

where $x$ is our sample value, $\mu$ is the population mean, and $\sigma$ is the population standard deviation.

__Task__: compute the z-score for every brain in our dataset, and for the sample brain, and display them.

The distribution of values once z-scored is called the <u>*standard* normal distribution</u>, and it has a mean of 0 and a standard deviation of 1. 

We can use this system to express in general terms (a) how much of an outlier a data point is (ex. "1.2 standard deviations from the mean"), and (b) how likely that data point was to have arisen from the population (p-value).

We can use existing packages in Python to determine the probability of observing any given data point in a normal distribution. Here is an example:

In [44]:
from scipy.stats import norm # norm is the normal distribution

z_score = (sample_brain_mass-mean_human_brain_mass) / np.std(brain_masses, ddof=1)
p = norm.pdf(z_score) # probability density function: returns the fraction of the distribution that is greater than the input value 
# (and it uses the standard normal distribution by default)
print(p)

# mention here the string formatting of floats

0.07889262466498656


### Putting it all together: conducting a hypothesis test

__Null hypothesis__: our sample brain comes from the population of human brains  
__Alternative hypothesis__: our sample brain comes from a separate population of brains  

__Compute z-score__  
__Compute p-value__  
__Compute effect size__ 

give a brief primer on effect size

todo: what are the reasonable conclusions that we can make? interpreting the pvalue, etc.

*Important note*: this statistical analysis did *not* help us with the fact that we assumed mass measurements alone can identify the source of a brain!

This is also quite an odd scenario in reality, because we rarely have information about full *populations* and single data points to compare them to.

---

## Part 2: dealing with samples of multiple observations

We often want to know about a *group of sampled points* as opposed to a single point.  
A common example would be to ask "does this sample come from a population with a mean of X?" 

This has a similar flavor to our first example question. It's a bit more complex, but the same logic applies.

Let's begin by loading in a similar dataset, but this time consisting of a set of sampled observations.  
Suppose each sample represents the mean firing rate of a single neuron.  

In [46]:
sample = np.load('sample.npy')

print(sample)

[ 19.08765187  68.65806486  33.56354929  32.49577301  30.48491205
   9.27890057  27.96165612  -3.17118488  19.24172174  35.57012565
   3.46882322  33.69680934  29.00681338  26.24718219  34.13053343
  82.60800516  72.86523956  63.62233261  53.87813344  -2.98364416
   2.923537    52.54128835   9.43490808  61.32799195   4.09816448
  88.87137507  36.40587655  46.21497085 -19.43758789  20.50959796
 -44.17382065  22.16441264  -1.10324205 -13.17386638  42.25494227
  27.17121254  94.43270511 -41.27862449  35.76603664  42.72439828
 -17.77928443 -58.33710308  42.46037535  51.20356776  65.45432975
  37.14975646   0.503567    54.79503172  34.73825892 -24.52529967
  13.70100255  14.9085       4.30102928 -26.74032759  63.57866916
 100.64613485  53.43042129  59.48199558 -22.85693732  13.48268966
 -27.91227646  -0.54621774  56.90457851  24.52251321  58.36478292
  49.79139193  38.61330352  36.20647124  -0.23270032  55.51730976
   4.61814591  74.60904414   6.19978153  24.11885351 -28.61337162
  -1.32663

We suspect that these neurons come from a neuronal population that has a mean firing rate of 30 Hz.

Null hypothesis: the sample comes from a population with a mean of 30.  
Alternative hypothesis: the sample comes from a population with a mean not equal to 30.

__Task__: inspect these data and speculate on how you might answer this question.

In [7]:
# inspect here
print(np.mean(sample))

27.030809250655714


*A naïve idea:* compute the sample mean, and if it differs from 30, deem the alternative hypothesis correct.

This and many other simple approaches don't work because we don't know __what to expect by chance when sampling a set of points from a population__. In other words, we haven't specified our *null distribution* (i.e. the one we conveniently had available in the previous example).

In [49]:
# visual demonstration

xs = np.arange(15,45,.01)
ys = norm.pdf(xs, 30, 5)
pl.plot(xs, ys, color='black', label='Population')
pl.axvline(30, lw=4, color='k')
pl.legend(loc='upper left')
pl.waitforbuttonpress()
pl.twinx()

for i,col in enumerate(['maroon','darkorange','forestgreen','steelblue','violet','grey']):
    sample = np.random.normal(30, 5, size=20)
    pl.hist(sample, bins=8, density=True, histtype='step', color=col, alpha=.5, label=f'Sample {i}')
    pl.axvline(sample.mean(), lw=2, color=col)
    pl.legend(loc='upper right')
    pl.waitforbuttonpress()

To figure this out, we could look into probability or statistics theory, but we can also build some intuition ourselves first.

Let's create a "known" population and sample from it many times, then analyze what we observe.

This will allow us to build some intuition about what happens when we sample, and what to expect about a population based on a single sample that came from it.

In [50]:
mean,std = 30,10

population = np.random.normal(mean,std,size=100000)
# show this histogram, and say: if we can somehow infer this exact population properties using just a single sample,
# then we can do our original test -- that's what a one-sample t-test ends up being
pl.hist(population, color='k', histtype='step', bins=200, density=True)
xs = np.arange(20,40,.01)
ys = norm.pdf(xs,mean,std)
pl.plot(xs,ys,color='k')
pl.title(f'Our "population": mean={mean}, std={std}')

Text(0.5,1,'Our "population": mean=30, std=10')

We want to learn what we can expect when sampling from this population.

todo: note "with/without replacement" - could be a chance to define replacement

__Task:__ from our "population", collect 10000 random samples each containing 50 observations.

todo: be extremely clear about samples vs observations vs etc

In [13]:
N = 10000 # number of times to sample the population
n = 50 # number of observations to take in each sample
samples = np.array([np.random.choice(population, size=n) for i in range(N)])

Lesson #1 of sampling: how do the means of our samples relate to mean of the population?

In [57]:
fig,axs = pl.subplots(2,1,sharex=True,gridspec_kw=dict(hspace=.4))
cols = ['red','orange','gold','green','blue','violet','grey','steelblue','pink']

pl.waitforbuttonpress()

for si,(sample,col) in enumerate(zip(samples,cols)):
    axs[0].hist(sample, histtype='step', color=col)
    axs[0].axvline(sample.mean(), color=col)
    axs[1].axvline(sample.mean(), color=col, alpha=.5)
    axs[0].set_xlabel('Observed value')
    axs[0].set_ylabel('Frequency')
    axs[0].set_title('Each histogram is a sample of observations\nWe get one of these when we do an experiment in real life')
    axs[1].set_xlabel('Mean of observations in a given sample')
    pl.pause(1)
axs[1].hist(samples.mean(axis=1), bins=50, histtype='step', color='k', lw=3)
axs[1].set_ylabel('Frequency')

Text(91.5694,0.5,'Frequency')

So now we're beginning to get a sense of how much we can expect our sample mean to reflect the true mean.

This distribution is our "null distribution."

todo: comment on specific details: is it symmetric? etc.

If we could characterize the center and shape of our null distribution (i.e. its mean and standard deviation), we could compute the probability of observing a single sample from it, as we did in the simpler case before.

Lesson #2 of sampling: how does `n` of our samples relate to our estimate of the population mean?

In [58]:
#task

for n in range(10,200,50):
    samples = np.array([np.random.choice(population, size=n) for i in range(N)])
    axs[1].hist(samples.mean(axis=1), bins=50, histtype='step', label=f'n={n}', lw=3, color=pl.cm.Greys(n/200+.1))
#pl.xlabel('Mean of observations in a given sample')
#pl.ylabel('Frequency')
axs[1].legend(title='n: # of observations in a sample', fontsize='x-small')

Lesson #3 of sampling: How can we use one sample to infer our null distribution?

It turns out that our null distribution of sample means can be inferred from:  
* (1) our sample mean
* (2) our sample standard deviation ($s$)
* (3) our sample size ($n$)

Specifically, our null distribution is estimated as a normal distribution with the same mean as our sample, and a standard deviation equal to $\frac{s}{\sqrt n}$.  
This latter fraction is called the __standard error of the mean__, or SEM.

Now we can ask our original question, rephrasing it as: __How much of an outlier is the *difference between our sample mean and the hypothesized mean of 30*?__  

And we are now able to precisely specify how we expect this *difference* to be distributed:  

In [61]:
from scipy.stats import t

for n,col in zip([5,10,1000],['r','g','b']):
    samples = np.array([np.random.choice(population, size=n) for i in range(N)])
    normed = (samples.mean(axis=1)-mean) / (samples.std(axis=1,ddof=1)/np.sqrt(n))
    pl.hist(normed, bins=100, histtype='step', label=f'n={n}', density=True, lw=1, color=col)
    
    xs = np.arange(-4,4,.01)
    ys = t.pdf(xs, n-1)
    pl.plot(xs,ys, color=col)
pl.legend(fontsize='x-small')
pl.xlim([-5,5])
pl.title('The t-distribution')

Text(0.5,1,'The t-distribution')

__Task__: load in the sample from above and compute a manual t-test to determine whether or not it comes from a population with a mean of 30.

sample = np.load('sample.npy')

now realize that we can build any distribution by asking the right question. in this case, our question is about the distribution of the z-scored sample1-sample2 value taken from 2 distinct populations. take for granted how we zscore it for a moment, and look at the distribution. (draw empirical t-distribution)

so we do the exact same thing: we compute our normalized score, compare to the distribution, and figure out where we sit...

now, importantly, what's the key thing here? is it the p value, or perhaps just the distance from the center?

show examples of how n will skew p values

stop here to note that this concept covers *ALL* (except maybe bayesian) hypothesis testing- comparing what you see to a distribution

now draw 2 generalized distributions and annotate with definitions of: effect size, pvalue, power, confidence intervals

okay, but we have a problem. all of this assumed a specific shape of distribution! what if that's not true?

there exist other tests that operate very similarly: "non-parametric," and even homebrew ones